In [1]:
import pandas as pd

# Prep Car dataset

In [2]:
relevant_cols = ['Model',
 'Gas Mileage',
 'Engine',
 'EPA Class',
 'Style Name',
 'Drivetrain', 
 'Displacement',
'Hybrid/Electric Components Note',
 'Year',
 'Price',
 'Torque (Nm)',
 'Max Torque (rpm)',
 'Power (kW)',
 'Max Power (rpm)']

df_cars = pd.read_parquet("./data/car_data_raw.parquet")[relevant_cols]

df_cars["Cylinders"] = df_cars["Engine"].str.extract("[V,I]-(\d{1,2}).*").astype(float).fillna(0)
df_cars["Displacement (liter)"] = df_cars["Displacement"].str.extract("(\d{1,2}.\d+)\s+L").astype(float)
df_cars["Fuel_efficiency"] = 235.21/df_cars["Gas Mileage"].str.lower().str.strip().str.extract("(\d+) mpg city\/(\d+) mpg hwy").astype(float).mean(axis=1)
df_cars["Torque (Nm)"] = df_cars["Torque (Nm)"].astype(float)
df_cars["Power (kW)"] = df_cars["Power (kW)"].astype(float)
df_cars = df_cars[df_cars["Torque (Nm)"].rank(pct=True) < 0.99].dropna(subset=["Cylinders"])
df_cars["Cylinders"] = df_cars["Cylinders"].astype(int)
df_cars["has_turbo"] = df_cars["Engine"].str.lower().str.contains("turbo")
df_cars["has_supercharger"] = df_cars["Engine"].str.lower().str.contains("supercharger")
df_cars["forced_induction"] = df_cars["has_turbo"] | df_cars["has_supercharger"]
df_cars["Drivetrain"] = df_cars["Drivetrain"].str.strip().str.lower().str.replace("-", " ").str.replace("4", "four")
df_cars["is_diesel"] = df_cars.Engine.str.contains("Diesel")
df_cars = df_cars[df_cars["Drivetrain"].isin(["front wheel drive", "rear wheel drive", "all wheel drive", "four wheel drive"])]


In [3]:
df_cars.to_parquet("./data/car_data_prepd.parquet")

# Prep Vacc dataset

In [4]:
df_vacc_del = pd.read_csv("data/germany_deliveries_timeseries_v2.tsv", 
                       sep="\t",
                       parse_dates=[0], 
                       header=0,
                       names=["date", "type_vacc", "region", "amount"]).set_index("date")
df_vacc_del = df_vacc_del[~df_vacc_del["type_vacc"].str.contains("BUND")]
df_vacc_del["date"] = df_vacc_del.index

In [5]:
df_vacc_del["week_year"] = df_vacc_del["date"].dt.isocalendar().year.astype(str) + "_" + df_vacc_del["date"].dt.isocalendar().week.astype(str).str.pad(2,fillchar='0')
df_vacc_del = df_vacc_del.groupby(["region", "type_vacc", "week_year"])[["amount"]].sum().reset_index(drop=False)
df_vacc_del.to_parquet("./data/vaccine_data_prepd.parquet")